In [38]:
from qiskit_aer import AerSimulator, Aer
from qiskit_ibm_runtime.fake_provider import FakePerth
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import Statevector, partial_trace, DensityMatrix 
from qiskit_experiments.framework import ParallelExperiment
from qiskit_experiments.library import StateTomography
from qiskit.visualization import plot_state_city
import qutip as qt
import numpy as np

backend = Aer.get_backend('aer_simulator_density_matrix')

In [39]:
nq = 5
anc = 1
ghz_state = QuantumCircuit(nq)
ghz_state.h(0)
ghz_state.s(0)
for i in range(1, nq):
    ghz_state.cx(0, i)
ghz_state = Statevector(ghz_state)
ghz_dm = DensityMatrix(ghz_state)

# GHZ State preparation circuit

qr_sys = QuantumRegister(nq, 'q')
qr_anc = QuantumRegister(anc, 'anc')
cr_anc = ClassicalRegister(anc, 'c')
qc_ghz = QuantumCircuit(qr_anc, qr_sys, cr_anc)
qc_ghz.h(0)
qc_ghz.s(0)
for i in range(1, nq):
    qc_ghz.cx(0, i)

qc_ghz.measure(qr_anc, cr_anc)
qc_ghz.reset(qr_anc)

# QST Experiment
qstexp1 = StateTomography(qc_ghz)
qstdata1 = qstexp1.run(backend, shots=1000).block_for_results()  #? Did this project to 0 for the ancilla, or traced it out?

# fid_result = qstdata1.analysis_results("state_fidelity")
# print("State Fidelity = {:.5f}".format(fid_result.value))

In [41]:
# Print results
# for result in qstdata1.analysis_results():
#     print(result)

state_from_qst = qstdata1.analysis_results("state").value
reduced_state_qst = partial_trace(state_from_qst, [0])
reduced_state_qst /= np.trace(reduced_state_qst)

fid_with_reduced_qt = qt.fidelity(qt.Qobj(ghz_state.data), qt.Qobj(reduced_state_qst.data))
print("Fidelity of GHZs from reduced DM of QST state = {:.5f}".format(fid_with_reduced_qt))

zero_projector = qt.tensor([qt.qeye(2**nq), qt.basis(2, 0).dag()]).full()
projected_state_qst = zero_projector @ state_from_qst.data @ zero_projector.T.conj()
projected_state_qst /= np.trace(projected_state_qst)

fid_with_projected_qt = qt.fidelity(qt.Qobj(ghz_state.data), qt.Qobj(projected_state_qst))
print("Fidelity of projected QST state to 0/1 measurement result= {:.5f}".format(fid_with_projected_qt))

zero_padded_ghz = qt.tensor([qt.Qobj(ghz_dm.data), qt.ket2dm(qt.basis(2, 0))]).full()
zero_padded_ghz /= np.trace(zero_padded_ghz)
fid_with_padded_qt = qt.fidelity(qt.Qobj(zero_padded_ghz), qt.Qobj(state_from_qst.data))
print("Fidelity of padded GHZ state with QST with 0/1 ancilla state result = {:.5f}".format(fid_with_padded_qt))


traced_out_system = partial_trace(state_from_qst, [1, 2, 3, 4, 5]).data
print('Ancilla state traced out:')
print(traced_out_system)

Fidelity of GHZs from reduced DM of QST state = 0.48947
Fidelity of projected QST state to 0/1 measurement result= 0.49232
Fidelity of padded GHZ state with QST with 0/1 ancilla state result = 0.48827
Ancilla state traced out:
[[0.98363109-1.49223714e-18j 0.00289918-1.21095118e-03j]
 [0.00289918+1.21095118e-03j 0.01636891+5.80890203e-20j]]
